In [1]:
%pip install --quiet --upgrade numpy pandas

import re
import numpy as np
import pandas as pd

# try the relative path first
df_raw = pd.read_csv(r"..\..\data\Stella\33101048.csv", dtype="string", encoding="utf-8-sig")
df_raw.head()


Note: you may need to restart the kernel to use updated packages.


,REF_DATE,GEO,DGUID,Business characteristics,Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2025,Canada,2021A000011124,North American Industry Classification System ...,Train current staff to use artificial intellig...,Percent,239,units,0,v1775199489,1.1.1,49.8,B,<NA>,<NA>,1
1,2025,Canada,2021A000011124,North American Industry Classification System ...,Hire staff trained in artificial intelligence ...,Percent,239,units,0,v1775199491,1.1.2,12.6,A,<NA>,<NA>,1
2,2025,Canada,2021A000011124,North American Industry Classification System ...,Purchase computing power or specialized equipment,Percent,239,units,0,v1775199492,1.1.3,14.2,A,<NA>,<NA>,1
3,2025,Canada,2021A000011124,North American Industry Classification System ...,Purchase cloud services or cloud storage,Percent,239,units,0,v1775199493,1.1.4,27.0,B,<NA>,<NA>,1
4,2025,Canada,2021A000011124,North American Industry Classification System ...,Change data collection or data management prac...,Percent,239,units,0,v1775199494,1.1.5,25.6,B,<NA>,<NA>,1


In [2]:
# --- 1) Filter to Canada, 2025 (Q3 2025 release is in REF_DATE=2025 for this table) ---
df = df_raw.copy()
df = df[(df["REF_DATE"].astype(str) == "2025") & (df["GEO"] == "Canada")].copy()

# --- 2) Rename columns to simple, analysis-friendly names ---
df = df.rename(columns={
    "Business characteristics": "category",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services": "change",
    "VALUE": "value_pct",
    "STATUS": "quality",
    "SYMBOL": "symbol"
})

# Ensure numeric type for the percentage
df["value_pct"] = pd.to_numeric(df["value_pct"], errors="coerce")

# --- 3) Add quality flags (StatsCan legend: E = use with caution; F = too unreliable) ---
df["flag_unreliable"] = (df["symbol"] == "F") | (df["quality"] == "F")
df["flag_caution"]    = (df["symbol"] == "E") | (df["quality"] == "E")

# --- 4) Optional: enforce a consistent display/order for the 10 change options ---
CHANGE_COLS = [
    "Train current staff to use artificial intelligence (AI)",
    "Hire staff trained in artificial intelligence (AI)",
    "Purchase computing power or specialized equipment",
    "Purchase cloud services or cloud storage",
    "Change data collection or data management practices",
    "Develop new workflows",
    "Use vendors or consulting services to install or integrate artificial intelligence (AI)",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, other change",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, unknown",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, none",
]
df["change"] = pd.Categorical(df["change"], categories=CHANGE_COLS, ordered=True)

# --- 5) Keep only analysis-critical columns (you can always bring others back) ---
df = df[["category", "change", "value_pct", "quality", "symbol", "flag_caution", "flag_unreliable", "UOM"]]

display(df.head(10))
print("Rows:", len(df), "| categories:", df["category"].nunique(), "| change options:", df["change"].nunique())


,category,change,value_pct,quality,symbol,flag_caution,flag_unreliable,UOM
0,North American Industry Classification System ...,Train current staff to use artificial intellig...,49.8,B,<NA>,<NA>,<NA>,Percent
1,North American Industry Classification System ...,Hire staff trained in artificial intelligence ...,12.6,A,<NA>,<NA>,<NA>,Percent
2,North American Industry Classification System ...,Purchase computing power or specialized equipment,14.2,A,<NA>,<NA>,<NA>,Percent
3,North American Industry Classification System ...,Purchase cloud services or cloud storage,27.0,B,<NA>,<NA>,<NA>,Percent
4,North American Industry Classification System ...,Change data collection or data management prac...,25.6,B,<NA>,<NA>,<NA>,Percent
5,North American Industry Classification System ...,Develop new workflows,41.9,B,<NA>,<NA>,<NA>,Percent
6,North American Industry Classification System ...,Use vendors or consulting services to install ...,16.0,A,<NA>,<NA>,<NA>,Percent
7,North American Industry Classification System ...,Changes business or organization will make whe...,0.2,A,<NA>,<NA>,<NA>,Percent
8,North American Industry Classification System ...,Changes business or organization will make whe...,14.0,A,<NA>,<NA>,<NA>,Percent
9,North American Industry Classification System ...,Changes business or organization will make whe...,12.3,A,<NA>,<NA>,<NA>,Percent


Rows: 670 | categories: 67 | change options: 10


In [5]:
import re
import pandas as pd
import numpy as np

# --- 1) Map each "change" to a strategic pillar ---
PILLAR_MAP = {
    "Train current staff to use artificial intelligence (AI)": "Skills",
    "Hire staff trained in artificial intelligence (AI)": "Skills",
    "Purchase computing power or specialized equipment": "Infra",
    "Purchase cloud services or cloud storage": "Infra",
    "Change data collection or data management practices": "Data/Workflow",
    "Develop new workflows": "Data/Workflow",
    "Use vendors or consulting services to install or integrate artificial intelligence (AI)": "External Help",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, other change": "Other",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, unknown": "Unknown",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, none": "None",
}
df["pillar"] = df["change"].map(PILLAR_MAP)

# Helper flags
df["is_proactive"] = df["pillar"].isin(["Skills","Infra","Data/Workflow","External Help"])
df["is_unknown"]   = df["pillar"].eq("Unknown")
df["is_none"]      = df["pillar"].eq("None")

# --- 2) Classify "category" into group/subgroup with short labels (built for your pasted StatsCan table) ---
INDUSTRIES = {
    "North American Industry Classification System (NAICS), all industries": "All industries",
    "Agriculture, forestry, fishing and hunting": "Agriculture",
    "Mining, quarrying, and oil and gas extraction": "Mining & O&G",
    "Construction": "Construction",
    "Manufacturing": "Manufacturing",
    "Wholesale trade": "Wholesale",
    "Retail trade": "Retail",
    "Transportation and warehousing": "Transport & warehousing",
    "Information and cultural industries": "Information & culture",
    "Finance and insurance": "Finance & insurance",
    "Real estate and rental and leasing": "Real estate & leasing",
    "Professional, scientific and technical services": "Prof., sci. & tech",
    "Administrative and support, waste management and remediation services": "Admin & waste mgmt",
    "Health care and social assistance": "Health & social",
    "Arts, entertainment and recreation": "Arts & recreation",
    "Accommodation and food services": "Accommodation & food",
    "Other services (except public administration)": "Other services",
}
SIZES = {
    "Business or organization size of employment, all employment sizes": "All sizes",
    "1 to 4 employees": "1–4",
    "5 to 19 employees": "5–19",
    "20 to 99 employees": "20–99",
    "100 or more employees": "100+",
}
TYPES = {
    "Business or organization type, all businesses or organizations": "All types",
    "Government agencies": "Government",
    "Private sector businesses": "Private",
    "Non-profit organizations": "Non-profit (all)",
    "Non-profit organizations serving households or individuals": "Non-profit (HH/individuals)",
    "Non-profit organizations serving businesses": "Non-profit (businesses)",
}
AGES = {
    "Age of business or organization, all ages": "All ages",
    "Age of business or organization, 2 years or less": "≤2 years",
    "Age of business or organization, 3 to 10 years old": "3–10 years",
    "Age of business or organization, 11 to 20 years old": "11–20 years",
    "Age of business or organization, more than 20 years old": ">20 years",
}
GEOS = {
    "Geography, all geography": "All geography",
    "Urban": "Urban",
    "Rural": "Rural",
}
OWNERSHIP_MAJ = {
    "Majority ownership, all ownerships": "All ownerships",
    "Majority ownership, woman": "Majority: women",
    "Majority ownership, First Nations, Métis or Inuit": "Majority: Indigenous",
    "Majority ownership, immigrant to Canada": "Majority: immigrants",
    "Majority ownership, person with a disability": "Majority: disability",
    "Majority ownership, LGBTQ2 people": "Majority: LGBTQ2",
    "Majority ownership, visible minority": "Majority: visible minority",
}
OWNERSHIP_VIS = {
    "Ownership by visible minority, all visible minorities": "Visible minority (all)",
    "Ownership by visible minority, South Asian": "South Asian",
    "Ownership by visible minority, Chinese": "Chinese",
    "Ownership by visible minority, Black": "Black",
    "Ownership by visible minority, Filipino": "Filipino",
    "Ownership by visible minority, Latin American": "Latin American",
    "Ownership by visible minority, Arab": "Arab",
    "Ownership by visible minority, Southeast Asian": "Southeast Asian",
    "Ownership by visible minority, West Asian": "West Asian",
    "Ownership by visible minority, Korean": "Korean",
    "Ownership by visible minority, Japanese": "Japanese",
    "Ownership by visible minority, other visible minority": "Other visible minority",
    "Ownership by visible minority, preferred not to say": "Preferred not to say",
}
ACTIVITY = {
    "Business or organization activity in the last 12 months, all business or organization activities": "All activities",
    "Exported goods outside of Canada": "Exported goods",
    "Exported services outside of Canada": "Exported services",
    "Made investments outside of Canada": "Invested outside Canada",
    "Sold goods to businesses in Canada who then resold them outside of Canada": "Sold to Canadian exporters",
    "Imported goods from outside of Canada": "Imported goods",
    "Imported services from outside of Canada": "Imported services",
    "Relocated any business or organizational activities or employees from another country into Canada": "Relocated into Canada",
    "Relocated any business or organizational activities or employees from Canada to another country": "Relocated out of Canada",
    "Engaged in other international business activities": "Other international activity",
    "Business or organization activity, none or other": "None/other activity",
}

def classify_category(cat):
    c = str(cat)
    # ordered checks: the "all ..." rows contain the group phrase
    if c in INDUSTRIES or c.startswith("North American Industry Classification System"):
        # normalize "all industries" row
        subgroup = INDUSTRIES.get(c, "All industries") if "all industries" in c else INDUSTRIES.get(c, c)
        return "Industry", subgroup
    if c in SIZES or c.startswith("Business or organization size of employment"):
        subgroup = SIZES.get(c, "All sizes")
        return "Size", subgroup
    if c in TYPES or c.startswith("Business or organization type"):
        subgroup = TYPES.get(c, "All types")
        return "Type", subgroup
    if c in AGES or c.startswith("Age of business or organization"):
        subgroup = AGES.get(c, "All ages")
        return "Age", subgroup
    if c in GEOS or c.startswith("Geography"):
        subgroup = GEOS.get(c, "All geography")
        return "Geography", subgroup
    if c in OWNERSHIP_MAJ or c.startswith("Majority ownership"):
        subgroup = OWNERSHIP_MAJ.get(c, c.replace("Majority ownership, ", "Majority: "))
        return "Ownership (majority)", subgroup
    if c in OWNERSHIP_VIS or c.startswith("Ownership by visible minority"):
        subgroup = OWNERSHIP_VIS.get(c, c.replace("Ownership by visible minority, ", "Visible minority: "))
        return "Ownership (visible minority)", subgroup
    if c in ACTIVITY or c.startswith("Business or organization activity in the last 12 months"):
        subgroup = ACTIVITY.get(c, "All activities")
        return "Activity (last 12 mo)", subgroup
    # fallback: treat as Industry name if it looks like one of the long NAICS names
    return "Unclassified", c

group_sub = df["category"].apply(classify_category)
df["group"]    = [g for g, s in group_sub]
df["subgroup"] = [s for g, s in group_sub]

# --- 3) Compute story-ready signals per subgroup ---
def summarize_group(df_in, group_name):
    d = df_in[df_in["group"] == group_name].copy()
    # pivot pillar -> pct
    piv = d.pivot_table(index="subgroup", columns="pillar", values="value_pct", aggfunc="sum", fill_value=0)

    # ensure all pillar columns exist
    for p in ["Skills","Infra","Data/Workflow","External Help","Other","Unknown","None"]:
        if p not in piv.columns: piv[p] = 0.0

    piv["Proactive (sum)"] = piv[["Skills","Infra","Data/Workflow","External Help"]].sum(axis=1)
    piv["Skills bias"]     = piv["Skills"] - (piv["Infra"])  # >0 means skills-first; <0 infra-first
    piv["Data/Workflow %"] = piv["Data/Workflow"]
    piv["Unknown %"]       = piv["Unknown"]
    piv["None %"]          = piv["None"]

    # top action (within proactive pillars)
    def top_action_for(sub):
        rows = d[(d["subgroup"] == sub) & (d["is_proactive"])]
        if rows.empty:
            return np.nan
        top_idx = rows["value_pct"].idxmax()
        return rows.loc[top_idx, "change"]

    piv["Top proactive action"] = [top_action_for(sg) for sg in piv.index]

    # helpful sorts for scanning
    quick = piv.reset_index().sort_values(["Unknown %","None %","Proactive (sum)"], ascending=[False, False, True])

    keep_cols = ["subgroup","Proactive (sum)","Skills","Infra","Data/Workflow","External Help",
                 "Skills bias","Unknown %","None %","Top proactive action"]
    return quick[keep_cols]

quickscan_industry = summarize_group(df, "Industry")

print("Quickscan — Industry (where confusion/inaction may be highest)")
display(quickscan_industry.head(12))

print("\nTip: Look for high 'Unknown %' or 'None %' with low 'Proactive (sum)'. Those are prime targets for a story + solution.")


Quickscan — Industry (where confusion/inaction may be highest)


pillar,subgroup,Proactive (sum),Skills,Infra,Data/Workflow,External Help,Skills bias,Unknown %,None %,Top proactive action
0,All industries,187.1,62.4,41.2,67.5,16.0,21.2,14.0,12.3,Train current staff to use artificial intellig...



Tip: Look for high 'Unknown %' or 'None %' with low 'Proactive (sum)'. Those are prime targets for a story + solution.


In [6]:
# Uses your df from Step 2 (already filtered to REF_DATE=2025 & GEO=Canada)
# 1) Focus on "all industries" row for the Industry group
all_industries_mask = df["category"].str.contains("all industries", case=False, na=False)
d = df[all_industries_mask].copy()

# 2) Pull the exact values we used
def get_pct(change_name):
    m = d[d["change"] == change_name]
    return float(m["value_pct"].iloc[0]) if not m.empty else float("nan")

train_staff = get_pct("Train current staff to use artificial intelligence (AI)")
hire_staff  = get_pct("Hire staff trained in artificial intelligence (AI)")
compute     = get_pct("Purchase computing power or specialized equipment")
cloud       = get_pct("Purchase cloud services or cloud storage")
data_mgmt   = get_pct("Change data collection or data management practices")
workflows   = get_pct("Develop new workflows")
vendors     = get_pct("Use vendors or consulting services to install or integrate artificial intelligence (AI)")
other_chg   = get_pct("Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, other change")
unknown     = get_pct("Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, unknown")
none        = get_pct("Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, none")

proactive_sum = train_staff + hire_staff + compute + cloud + data_mgmt + workflows + vendors
data_workflow_sum = data_mgmt + workflows

print({
    "Train staff": train_staff,
    "Hire staff": hire_staff,
    "Compute (hardware)": compute,
    "Cloud": cloud,
    "Change data mgmt": data_mgmt,
    "Develop workflows": workflows,
    "Vendors/consulting": vendors,
    "Other change": other_chg,
    "Unknown": unknown,
    "None": none,
    "Proactive (sum)": proactive_sum,
    "Data/Workflow (sum)": data_workflow_sum
})


{'Train staff': 49.8, 'Hire staff': 12.6, 'Compute (hardware)': 14.2, 'Cloud': 27.0, 'Change data mgmt': 25.6, 'Develop workflows': 41.9, 'Vendors/consulting': 16.0, 'Other change': 0.2, 'Unknown': 14.0, 'None': 12.3, 'Proactive (sum)': 187.1, 'Data/Workflow (sum)': 67.5}


In [7]:
# Reuse the pillar mapping from Step 3 (if you skipped it, re-run that cell first)
PILLAR_MAP = {
    "Train current staff to use artificial intelligence (AI)": "Skills",
    "Hire staff trained in artificial intelligence (AI)": "Skills",
    "Purchase computing power or specialized equipment": "Infra",
    "Purchase cloud services or cloud storage": "Infra",
    "Change data collection or data management practices": "Data/Workflow",
    "Develop new workflows": "Data/Workflow",
    "Use vendors or consulting services to install or integrate artificial intelligence (AI)": "External Help",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, other change": "Other",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, unknown": "Unknown",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, none": "None",
}
if "pillar" not in df.columns:
    df["pillar"] = df["change"].map(PILLAR_MAP)

# Classify categories into high-level groups & short labels (re-run if needed)
def classify_category(cat):
    c = str(cat)
    if c.startswith("North American Industry Classification System"):
        return "Industry", ("All industries" if "all industries" in c.lower() else c)
    if c.startswith("Business or organization size of employment"):
        return "Size", "All sizes"
    if c in {"1 to 4 employees","5 to 19 employees","20 to 99 employees","100 or more employees"}:
        return "Size", c
    if c.startswith("Business or organization type"):
        return "Type", "All types"
    if c.startswith("Non-profit") or c in {"Government agencies","Private sector businesses"}:
        return "Type", c
    if c.startswith("Age of business or organization"):
        return "Age", ("All ages" if "all ages" in c.lower() else c.split(", ",1)[1])
    if c.startswith("Geography"):
        return "Geography", "All geography"
    if c in {"Urban","Rural"}:
        return "Geography", c
    if c.startswith("Majority ownership"):
        return "Ownership (majority)", c.replace("Majority ownership, ","Majority: ")
    if c.startswith("Ownership by visible minority"):
        return "Ownership (visible minority)", c.replace("Ownership by visible minority, ","")
    if c.startswith("Business or organization activity in the last 12 months"):
        return "Activity (last 12 mo)", "All activities"
    if c.endswith("activity") or c.endswith("activities") or "Imported" in c or "Exported" in c or "Relocated" in c or "Sold goods" in c:
        return "Activity (last 12 mo)", c
    return "Unclassified", c

group_sub = df["category"].apply(classify_category)
df["group"]    = [g for g, s in group_sub]
df["subgroup"] = [s for g, s in group_sub]

# Build per-subgroup metrics
piv = df.pivot_table(index=["group","subgroup"], columns="pillar", values="value_pct", aggfunc="sum", fill_value=0.0)
for col in ["Skills","Infra","Data/Workflow","External Help","Unknown","None"]:
    if col not in piv.columns: piv[col] = 0.0

piv["Opportunity"] = piv["Skills"] + piv["Infra"] + piv["Data/Workflow"] + piv["External Help"]
piv["Risk"]        = piv["Unknown"] + piv["None"]

# Simple composite priority score (higher = more urgent equity attention)
# You can tweak weights; this just surfaces "stuck" places with low proactive movement.
piv["PriorityScore"] = piv["Risk"] - (piv["Opportunity"] / 200.0)

scores = piv.reset_index().sort_values(["group","PriorityScore"], ascending=[True, False])

display(scores.head(20))
print("Rows:", len(scores))


pillar,group,subgroup,Data/Workflow,External Help,Infra,None,Other,Skills,Unknown,Opportunity,Risk,PriorityScore
4,Activity (last 12 mo),Imported goods from outside of Canada,82.7,21.3,56.1,12.3,0.0,78.8,15.5,238.9,27.8,26.6055
0,Activity (last 12 mo),All activities,67.5,16.0,41.2,12.3,0.2,62.4,14.0,187.1,26.3,25.3645
2,Activity (last 12 mo),Exported goods outside of Canada,94.4,34.6,60.0,1.6,0.0,94.5,22.8,283.5,24.4,22.9825
3,Activity (last 12 mo),Exported services outside of Canada,0.0,14.0,7.3,7.7,0.0,74.2,4.5,95.5,12.2,11.7225
5,Activity (last 12 mo),Imported services from outside of Canada,85.5,21.8,57.9,2.6,0.0,91.4,7.3,256.6,9.9,8.617
1,Activity (last 12 mo),Engaged in other international business activi...,0.0,0.0,0.0,0.0,0.0,0.0,6.9,0.0,6.9,6.9
8,Activity (last 12 mo),Sold goods to businesses in Canada who then re...,23.1,6.4,13.2,1.4,0.0,85.1,3.3,127.8,4.7,4.061
7,Activity (last 12 mo),Relocated any business or organizational activ...,86.7,6.6,0.0,1.6,0.0,2.9,1.4,96.2,3.0,2.519
6,Activity (last 12 mo),Relocated any business or organizational activ...,23.7,2.9,14.6,0.0,0.0,9.5,0.0,50.7,0.0,-0.2535
10,Age,2 years or less,76.7,4.5,31.4,27.8,0.0,36.0,8.5,148.6,36.3,35.557


Rows: 67


In [8]:
import pandas as pd
import numpy as np

# --- Robust list of NAICS industry labels present in the StatsCan table ---
KNOWN_INDUSTRIES = {
    "North American Industry Classification System (NAICS), all industries": "All industries",
    "Agriculture, forestry, fishing and hunting": "Agriculture",
    "Mining, quarrying, and oil and gas extraction": "Mining & O&G",
    "Construction": "Construction",
    "Manufacturing": "Manufacturing",
    "Wholesale trade": "Wholesale",
    "Retail trade": "Retail",
    "Transportation and warehousing": "Transport & warehousing",
    "Information and cultural industries": "Information & culture",
    "Finance and insurance": "Finance & insurance",
    "Real estate and rental and leasing": "Real estate & leasing",
    "Professional, scientific and technical services": "Prof., sci. & tech",
    "Administrative and support, waste management and remediation services": "Admin & waste mgmt",
    "Health care and social assistance": "Health & social",
    "Arts, entertainment and recreation": "Arts & recreation",
    "Accommodation and food services": "Accommodation & food",
    "Other services (except public administration)": "Other services",
}

# --- Rebuild a reliable classifier (overwrites previous if any) ---
def classify_category_v2(cat: str):
    c = str(cat)

    # Industry (explicit list + the "all industries" long label)
    if c in KNOWN_INDUSTRIES or c.startswith("North American Industry Classification System"):
        lbl = KNOWN_INDUSTRIES.get(c, None)
        if lbl is None and "all industries" in c.lower():
            lbl = "All industries"
        elif lbl is None:
            lbl = c
        return "Industry", lbl

    # Size
    if c.startswith("Business or organization size of employment"):
        return "Size", "All sizes"
    if c in {"1 to 4 employees","5 to 19 employees","20 to 99 employees","100 or more employees"}:
        return "Size", c

    # Type
    if c.startswith("Business or organization type"):
        return "Type", "All types"
    if c in {"Government agencies","Private sector businesses",
             "Non-profit organizations",
             "Non-profit organizations serving households or individuals",
             "Non-profit organizations serving businesses"}:
        return "Type", c

    # Age
    if c.startswith("Age of business or organization"):
        if "all ages" in c.lower():
            return "Age", "All ages"
        # take the part after the comma for the specific age bucket
        return "Age", c.split(", ", 1)[-1]

    # Geography
    if c.startswith("Geography"):
        return "Geography", "All geography"
    if c in {"Urban","Rural"}:
        return "Geography", c

    # Ownership (majority)
    if c.startswith("Majority ownership"):
        return "Ownership (majority)", c.replace("Majority ownership, ", "Majority: ")

    # Ownership (visible minority)
    if c.startswith("Ownership by visible minority"):
        return "Ownership (visible minority)", c.replace("Ownership by visible minority, ", "")

    # Activity (last 12 mo)
    if c.startswith("Business or organization activity in the last 12 months"):
        return "Activity (last 12 mo)", "All activities"
    if any(k in c for k in ["Exported", "Imported", "Relocated", "Sold goods", "Other international", "Made investments"]):
        return "Activity (last 12 mo)", c

    # Fallback
    return "Unclassified", c

# Apply classifier v2
grp_sub = df["category"].apply(classify_category_v2)
df["group"]    = [g for g, s in grp_sub]
df["subgroup"] = [s for g, s in grp_sub]

# Pillar mapping (ensure present)
PILLAR_MAP = {
    "Train current staff to use artificial intelligence (AI)": "Skills",
    "Hire staff trained in artificial intelligence (AI)": "Skills",
    "Purchase computing power or specialized equipment": "Infra",
    "Purchase cloud services or cloud storage": "Infra",
    "Change data collection or data management practices": "Data/Workflow",
    "Develop new workflows": "Data/Workflow",
    "Use vendors or consulting services to install or integrate artificial intelligence (AI)": "External Help",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, other change": "Other",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, unknown": "Unknown",
    "Changes business or organization will make when using artificial intelligence (AI) to produce goods or deliver services, none": "None",
}
if "pillar" not in df.columns:
    df["pillar"] = df["change"].map(PILLAR_MAP)

# Aggregate by pillar for each subgroup
piv2 = df.pivot_table(index=["group","subgroup"], columns="pillar", values="value_pct", aggfunc="sum", fill_value=0.0)
for col in ["Skills","Infra","Data/Workflow","External Help","Unknown","None","Other"]:
    if col not in piv2.columns:
        piv2[col] = 0.0

# Opportunity/Risk + Priority score (higher = more urgent to intervene)
piv2["Opportunity"]   = piv2["Skills"] + piv2["Infra"] + piv2["Data/Workflow"] + piv2["External Help"]
piv2["Risk"]          = piv2["Unknown"] + piv2["None"]
piv2["PriorityScore"] = piv2["Risk"] - (piv2["Opportunity"] / 200.0)

scores2 = piv2.reset_index()

# --- Show INDUSTRY ranking to anchor your story ---
scores_industry = scores2[scores2["group"] == "Industry"].sort_values("PriorityScore", ascending=False)

print("Industry ranking (higher PriorityScore = more stuck/confused vs proactive):")
display(scores_industry)

# Sanity check: "All industries" row should match your earlier All industries numbers
print("\nSanity check — All industries (should show Opportunity≈187.1, Risk≈26.3):")
display(scores_industry[scores_industry["subgroup"] == "All industries"])


Industry ranking (higher PriorityScore = more stuck/confused vs proactive):


pillar,group,subgroup,Data/Workflow,External Help,Infra,None,Other,Skills,Unknown,Opportunity,Risk,PriorityScore
17,Industry,All industries,67.5,16.0,41.2,12.3,0.2,62.4,14.0,187.1,26.3,25.3645



Sanity check — All industries (should show Opportunity≈187.1, Risk≈26.3):


pillar,group,subgroup,Data/Workflow,External Help,Infra,None,Other,Skills,Unknown,Opportunity,Risk,PriorityScore
17,Industry,All industries,67.5,16.0,41.2,12.3,0.2,62.4,14.0,187.1,26.3,25.3645
